In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

In [ ]:
class FlexibleRNN(nn.Module):
    def __init__(
        self, input_size, hidden_size, output_size, num_layers, rnn_type="rnn"
    ):
        super(FlexibleRNN, self).__init__()

        # Choose the RNN cell type
        if rnn_type.lower() == "rnn":
            self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
        elif rnn_type.lower() == "lstm":
            self.rnn = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        elif rnn_type.lower() == "gru":
            self.rnn = nn.GRU(input_size, hidden_size, num_layers, batch_first=True)
        else:
            raise ValueError("Invalid rnn_type. Choose 'rnn', 'lstm', or 'gru'.")

        # Fully connected layer to map RNN output to the desired output size
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        # RNN forward pass
        rnn_out, _ = self.rnn(x)

        # Extract the output from the last time step
        last_output = rnn_out[:, -1, :]

        # Fully connected layer
        output = self.fc(last_output)

        return output

    def train_model(
        self, train_data, train_labels, num_epochs=300, learning_rate=0.001
    ):
        # Define loss function and optimizer
        criterion = nn.MSELoss()
        optimizer = optim.Adam(self.parameters(), lr=learning_rate)

        for epoch in range(num_epochs):
            # Forward pass
            outputs = self(train_data)
            loss = criterion(outputs, train_labels)

            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            if (epoch + 1) % 10 == 0:
                print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")

    def predict(self, input_data):
        # Make predictions for input_data
        with torch.no_grad():
            predictions = self(input_data)
        return predictions

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim


class TransformerModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_heads):
        super(TransformerModel, self).__init__()

        # MultiheadAttention layer
        self.attention = nn.MultiheadAttention(
            embed_dim=input_size, num_heads=num_heads
        )

        # Feedforward layer
        self.feedforward = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, output_size),
        )

    def forward(self, x):
        # Self-attention layer
        attention_out, _ = self.attention(x, x, x)

        # Feedforward layer
        output = self.feedforward(
            attention_out.mean(dim=1)
        )  # Aggregate over the sequence

        return output

    def train_model(
        self, train_data, train_labels, num_epochs=300, learning_rate=0.001
    ):
        # Define loss function and optimizer
        criterion = nn.MSELoss()
        optimizer = optim.Adam(self.parameters(), lr=learning_rate)

        for epoch in range(num_epochs):
            # Forward pass
            outputs = self(train_data)
            loss = criterion(outputs, train_labels)

            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            if (epoch + 1) % 10 == 0:
                print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")

    def predict(self, input_data):
        # Make predictions for input_data
        with torch.no_grad():
            predictions = self(input_data)
        return predictions

In [ ]:
# Example usage (RNNs for sequence processing):
# Define input size, hidden size, and output size
input_size = 10  # Number of expected features in the input
hidden_size = 20  # Number of features in the hidden state
output_size = 5  # Size of each output sample
num_layers = 2  # Number of recurrent layers

# Create an instance of the FlexibleRNN model
model = FlexibleRNN(input_size, hidden_size, output_size, num_layers)

# Create an instance of the FlexibleRNN model with LSTM
model_lstm = FlexibleRNN(
    input_size, hidden_size, output_size, num_layers, rnn_type="lstm"
)

# Create an instance of the FlexibleRNN model with GRU
model_gru = FlexibleRNN(
    input_size, hidden_size, output_size, num_layers, rnn_type="gru"
)

# Generate synthetic training data
train_data = torch.randn(100, 8, input_size)
train_labels = torch.randn(100, output_size)

# Train the model
model.train_model(train_data, train_labels)

# Train the LSTM model
model_lstm.train_model(train_data, train_labels)

# Train the GRU model
model_gru.train_model(train_data, train_labels)

# Generate a test sequence
test_data = torch.randn(1, 8, input_size)

# Make predictions
predictions = model.predict(test_data)
print("Predictions:", predictions)

# Make predictions with LSTM model
predictions_lstm = model_lstm.predict(test_data)
print("Predictions (LSTM):", predictions_lstm)

# Make predictions with GRU model
predictions_gru = model_gru.predict(test_data)
print("Predictions (GRU):", predictions_gru)

In [ ]:
# Example usage (transformer for sequence processing):
# Define input size, hidden size, and output size
input_size = 10   # Total dimension of the model (embed_dim)
hidden_size = 20  # Number of features in the hidden layers of feed-forward network
output_size = 5
num_heads = 2     # Number of parallel attention heads

# Create an instance of the TransformerModel
model_transformer = TransformerModel(input_size, hidden_size, output_size, num_heads)

# Generate synthetic training data
train_data = torch.randn(100, 8, input_size)
train_labels = torch.randn(100, output_size)

# Train the Transformer model
model_transformer.train_model(train_data, train_labels)

# Generate a test sequence
test_data = torch.randn(1, 8, input_size)

# Make predictions with Transformer model
predictions_transformer = model_transformer.predict(test_data)
print("Predictions (Transformer):", predictions_transformer)